In [42]:
import pandas as pd
import numpy as np

### KG

In [3]:
data=pd.read_csv('kg_v5_grouped.csv')

/Users/jingyiliu/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.head()

,Unnamed: 0,relation,x_id,x_type,x_name,x_source,y_id,y_type,y_name,y_source
0,0,protein_protein,9796,gene/protein,PHYHIP,NCBI,56992,gene/protein,KIF15,NCBI
1,1,protein_protein,7918,gene/protein,GPANK1,NCBI,9240,gene/protein,PNMA1,NCBI
2,2,protein_protein,8233,gene/protein,ZRSR2,NCBI,23548,gene/protein,TTC33,NCBI
3,3,protein_protein,4899,gene/protein,NRF1,NCBI,11253,gene/protein,MAN1B1,NCBI
4,4,protein_protein,5297,gene/protein,PI4KA,NCBI,8601,gene/protein,RGS20,NCBI


In [5]:
data['x_type'].value_counts()

gene/protein          2895758
drug                  2817799
anatomy               1810030
biological_process     251307
disease                219014
effect/phenotype       120391
molecular_function      95610
cellular_component      92444
pathway                 45700
exposure                10048
Name: x_type, dtype: int64

## Get the disease in the KG

In [6]:
disease_data=data[data['x_type']=='disease'][['x_id','x_name','x_type']]

In [7]:
disease_data.head()

,x_id,x_name,x_type
2995001,2816,adrenal cortex disease,disease
2995002,5340_4907_5339,alopecia areata\alopecia\androgenetic alopecia,disease
2995003,3847,Mendelian disease,disease
2995004,2243,hemorrhagic disease,disease
2995005,2245,blood platelet disease,disease


In [8]:
#get the number of unique disease
len(pd.unique(disease_data['x_name']))  #21392

21392

In [9]:
kg_uniquemondo=pd.DataFrame(disease_data.x_id.drop_duplicates())
kg_uniquemondo=kg_uniquemondo.rename(columns={"x_id":'Mondo'})
kg_uniquemondo=pd.DataFrame(kg_uniquemondo)

In [10]:
kg_uniquemondo.head()

,Mondo
2995001,2816
2995002,5340_4907_5339
2995003,3847
2995004,2243
2995005,2245


In [131]:
#pd.DataFrame(disease_data.x_name.drop_duplicates())

### Mapping table

In [11]:
#load the reference mapping table
map_data=pd.read_csv('mondo_references.csv')

In [12]:
map_data.head()

,Unnamed: 0,ontology_id,ontology,mondo_id
0,0,4,DOID,1
1,1,D004194,MESH,1
2,2,0000408,EFO,1
3,3,C0012634,UMLS,1
4,4,799.9,ICD9,1


In [14]:
#get the umls and mondo mapping table
umls_mondo=map_data[map_data['ontology']=='UMLS'][['ontology_id','mondo_id']]
umls_mondo=umls_mondo.rename(columns={"ontology_id":"CUI","mondo_id":'Mondo'})

In [15]:
umls_mondo.head()

,CUI,Mondo
3,C0012634,1
14,C0001623,4
23,C0405580,4
31,C0005818,9
37,C1285186,15


In [16]:
#sanity check
umls_mondo[umls_mondo['Mondo']==1]

,CUI,Mondo
3,C0012634,1


In [17]:
#sanity check, at the original obo file, some mondo id doesn't have corresponding umls id, 2245 is an example
umls_mondo[umls_mondo['Mondo']==2245]

,CUI,Mondo


### Load UMLS disease description

In [21]:
umls_data=pd.read_csv('umls_def_disorder.csv')

In [22]:
umls_data.head()

,Unnamed: 0,CUI,source,description
0,308,C0000744,MSH,An autosomal recessive disorder of lipid metab...
1,309,C0000744,CSP,disorder of lipid metabolism inherited as an a...
2,310,C0000744,NCI,An autosomal recessive disorder characterized ...
3,311,C0000744,HPO,An absence of low-density lipoprotein choleste...
4,314,C0000768,MSH,Malformations of organs or body parts during d...


In [20]:
#umls_uniquecui=pd.DataFrame(umls_data.CUI.drop_duplicates())

## Map KG disease and umls description

In [31]:
#umls inner join with mapping table
umls_mondo_map=pd.merge(umls_mondo, umls_data, on="CUI", how="inner")
umls_mondo_map=umls_mondo_map[['CUI','Mondo','source','description']]

In [32]:
umls_mondo_map.head()

,CUI,Mondo,source,description
0,C0012634,1,MSH,A definite pathologic process with a character...
1,C0012634,1,NCI_NCI-GLOSS,"In medicine, a health problem with certain cha..."
2,C0012634,1,NCI,Any abnormal condition of the body or mind tha...
3,C0012634,1,NCI_CDISC-GLOSS,Any deviation from or interruption of the norm...
4,C0012634,1,CSP,top term heading for all specific disorders an...


In [33]:
#csv file for kexin and payal
umls_mondo_map.to_csv('mondo_umls_description.csv',index=False)

### Mapping: calculate the mapping rate

In [37]:
#kg table:  unique mondo id
kg_uniquemondo.shape[0] #32530
kg_uniquemondo.head()

,Mondo
2995001,2816
2995002,5340_4907_5339
2995003,3847
2995004,2243
2995005,2245


In [35]:
#umls table: unique umls cui id
umls_uniquecui.shape[0] #20411
umls_uniquecui.head()

,CUI
0,C0000744
4,C0000768
7,C0000771
8,C0000772
11,C0000773


In [135]:
#mapping table
umls_mondo.shape[0] #22850
umls_mondo.head()

,UMLS,Mondo
3,C0012634,1
14,C0001623,4
23,C0405580,4
31,C0005818,9
37,C1285186,15


In [39]:
#There are 6151 umls description are able to map to the mapping table
df_merged=umls_uniquecui.merge(umls_mondo, how='inner', on='CUI')
df_merged.shape[0] #6151

6151

In [40]:
#after merge, extract those mondo ID
df_merged_mondo=pd.DataFrame(df_merged['Mondo'])
df_merged_mondo.head()

,Mondo
0,8692
1,43459
2,13727
3,4846
4,4846


In [159]:
df_merged_mondo.shape[0] #6150

6151

In [169]:
kg_uniquemondo.head()

,Mondo
2995001,2816
2995002,5340_4907_5339
2995003,3847
2995004,2243
2995005,2245


In [170]:
kg_uniquemondo.shape[0]

32530

In [43]:
a1=kg_uniquemondo['Mondo'].values #a1: unique kg mondo id
a1 = a1.astype(np.float)
a2=df_merged_mondo['Mondo'].values #a2: unique mondo id in the mapping table after mapping with umls
a2 = a2.astype(np.float)

In [44]:
len(np.intersect1d(a1, a2)) #5252 mondo in kg are able to map to umls

5252

In [45]:
5252/21392 #only 24.6% of the kg get mapped with umls description 

0.24551234106207928

In [199]:
kg_uniquemondo.head()

,Mondo
2995001,2816
2995002,5340_4907_5339
2995003,3847
2995004,2243
2995005,2245


In [47]:
#a3 reference; a1 kg
a3=umls_mondo['Mondo'].values #a1: unique mondo id in mapping table 
a3 = a3.astype(np.float)
len(np.intersect1d(a1, a3))  #only 17577 mondo id in kg has a reference in the mapping table

17577

In [49]:
no_ref = [item for item in a1 if item not in a3]
len(no_ref)  #5539 mondo id in kg doesn't have reference in mapping table

5539

In [50]:
5539/21392 #25.9% of kg mondo id doesn't have reference umls id in the mapping table

0.25892857142857145

In [208]:
#sanity check--> go back to mondo.obo file to check, they don't have umls reference
no_ref

[534049075339.0,
 2245.0,
 24300.0,
 542018612.0,
 20573.0,
 19824.0,
 1149.0,
 15979.0,
 19796.0,
 529881595387.0,
 15572.0,
 21147.0,
 15397.0,
 15503.0,
 15412.0,
 18562.0,
 6600.0,
 992286969555453.0,
 19695.0,
 100137.0,
 15707.0,
 20162.0,
 83452771.0,
 20285.0,
 5497.0,
 1602216021.0,
 21669.0,
 62.0,
 43005.0,
 15327.0,
 44975.0,
 100033.0,
 16072.0,
 15168.0,
 6504.0,
 37792.0,
 5677.0,
 6752.0,
 308.0,
 4.994504586472457e+223,
 20404678254.0,
 1673.0,
 10514795.0,
 24294.0,
 1920324295249.0,
 565715243.0,
 25294.0,
 5763.0,
 100120.0,
 5650.0,
 578418081.0,
 20067.0,
 21747.0,
 42488.0,
 5656.0,
 5745.0,
 255.0,
 45033.0,
 314.0,
 4.497289757145823e+19,
 4944927.0,
 341.0,
 21641.0,
 37938.0,
 16148.0,
 1.807660525861418e+34,
 5.447101083669182e+17,
 2415.0,
 1187498649874114.0,
 515030045417.0,
 20242.0,
 192102120.0,
 2176.0,
 21041.0,
 592.0,
 411.0,
 414.0,
 2069642976.0,
 5528.0,
 20606.0,
 18876409517597.0,
 21635.0,
 531453014515284.0,
 19213.0,
 20699.0,
 5476.0,
 207